In [ ]:
from __future__ import annotations

import eradiate
import matplotlib.pyplot as plt
from eradiate.constants import EARTH_RADIUS
from eradiate.units import unit_registry as ureg

eradiate.set_mode("mono_polarized")
eradiate.plot.set_style()

In [ ]:
def experiment(geometry, sza, img_res, timeout, integrator):
    if geometry == "plane_parallel":
        camera_origin = [1, -5, 1]
        camera_target = [0, 0, 1]
        camera_up = [0, 0, 1]

    elif geometry == "spherical_shell":
        camera_origin = [0, -5e-1, EARTH_RADIUS.m_as("km") + 1e-1] * ureg.km
        camera_target = [0, 0, EARTH_RADIUS.m_as("km") + 2e-1] * ureg.km
        camera_up = [0, 0, 1]

    else:
        raise ValueError

    return eradiate.experiments.AtmosphereExperiment(
        geometry=geometry,
        illumination={"type": "astro_object", "zenith": sza, "azimuth": 90},
        atmosphere={
            "type": "molecular",
            "phase": {"type": "rayleigh", "depolarization": 0.03},
        },
        surface={
            "type": "lambertian",
            "reflectance": {
                "type": "interpolated",
                "construct": "from_dataarray",
                "dataarray": eradiate.data.load_dataset(
                    "spectra/reflectance/lambertian_soil.nc"
                ).reflectance.sel(brightness="bright"),
            },
        },
        measures={
            "type": "perspective",
            "origin": camera_origin,
            "target": camera_target,
            "up": camera_up,
            "film_resolution": img_res,
            "rfilter": "box",
            "fov": 70.0,
            "sampler": "independent",
            "spp": 4,
            "srf": {
                "type": "delta",
                "wavelengths": [440, 550, 660],
            },
        },
        integrator={
            "type": integrator,
            "timeout": timeout,
            "stokes": True if eradiate.mode().is_polarized else False,
        },
    )

In [ ]:
SZA = 87

RESOLUTION = (256, 128)
# RESOLUTION = (512, 256)

SPP = 16
# SPP = 32
# SPP = 256

TIMEOUT = None

exp = experiment("spherical_shell", SZA, RESOLUTION, TIMEOUT, "volpath")
res = eradiate.run(exp, spp=SPP)
radiance = res.radiance

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(3 * 4, 3), layout="constrained")

rgb = eradiate.xarray.interp.dataarray_to_rgb(
    (radiance * 10).sel(stokes="I"),
    channels=[("w", 660), ("w", 550), ("w", 440)],
    normalize=False,
)
ax[0].imshow(rgb)
ax[0].set_axis_off()
ax[0].set_title("I - [440,550,660] nm")

w = 550
val = radiance.sel(stokes="Q", w=w).squeeze().values
ax[1].imshow(val, vmin=-0.01, vmax=0.01, cmap="RdBu_r")
ax[1].set_axis_off()
ax[1].set_title("Q - 550 nm")

val = radiance.sel(stokes="U", w=w).squeeze().values
ax[2].imshow(val, vmin=-0.002, vmax=0.002, cmap="RdBu_r")
ax[2].set_axis_off()
ax[2].set_title("U - 550 nm")

plt.show()
plt.close()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3), layout="constrained")

rgb = eradiate.xarray.interp.dataarray_to_rgb(
    (radiance * 10).sel(stokes="I"),
    channels=[("w", 660), ("w", 550), ("w", 440)],
    normalize=False,
)
ax.imshow(rgb)
ax.set_axis_off()
plt.savefig("../_images/polarized_sunset.png", bbox_inches="tight", pad_inches=0)